In [14]:
# Let's get all the PDFs eh! This takes ages - it's about 500mb for each period!

import requests
from bs4 import BeautifulSoup

pdfs = []

urls = {
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians-expenditure-P36/":"36",  
}    

for url in urls.keys():
    rep = requests.get(url)
    soup = BeautifulSoup(rep.content)
    links = soup.find_all("a")
    for l in links:
        try:
            if "pdf" in l.get("href"):
                pdfs.append(l.get("href"))
        except:
            pass



def download_file(url):
    downloaded = !cd pdfs;ls
    local_filename = url.split('/')[-1]
    if "30" in url:
        local_filename = "P30-"+local_filename
    if "31" in url:
        local_filename = "P31-"+local_filename
    if local_filename not in downloaded:
        # NOTE the stream=True parameter
        r = requests.get(url, stream=True)
        with open('pdfs/'+local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    #f.flush() commented by recommendation from J.F.Sebastian
        return local_filename
    
    
for pdf in pdfs:
    print download_file(pdf)

    

None
None
None
None
P36_SESELJA_Zed.pdf
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
P36_DANBY_Michael.pdf
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Non

In [15]:
# Next, let's check about all potential categories by looking at what terms are on the summary page - for all pdfs.

sumref = {} # This is to find the summary reference page for all pdfs

localpdfsbase = !cd pdfs;ls # Gets the filenames
localpdfs = []
for l in localpdfsbase:
    if "36" in l:
        localpdfs.append(l)

import pdfquery

def get_page(x,y): # this takes the pdfquery object and a list of pages to test (including None as catchall)
    for page in y:
        if page != None:
            x.load(page)
            titles = x.pq(":in_bbox('170.979,683.267,438.506,699.868')")
            for t in titles:
                if t.text.strip() == 'Summary of Parliamentary Expenditure by Period':
                    return int(page) # This returns the page
        else:
            x.load()
            tree = x.get_tree() # This is a last resort - search (and load!) the whole pdf to search for the summary
            for element in tree.iter():
                if element.text != '':
                    if element.text != None:
                        if element.text.strip() == 'Summary of Parliamentary Expenditure by Period':
                            for part in element.iterancestors(tag="LTPage"):
                                pageref = part.get("page_index")
                                return int(pageref)


# First, let's find the summary page for everyone
for pdffile in localpdfs:
    pdf = pdfquery.PDFQuery("pdfs/"+pdffile)
    sumref[pdffile] = get_page(pdf,[2,8,5,None]) # The most common pages are 2,8 and 5 - save time by trying these first

In [16]:
# This checks whether the total expenditure and the items added up are equal to each other.

import ast

import pandas as pd
from IPython.display import display
import collections                                         

def get_sum(x,y): #This takes the pdf and the page reference and produces summary stats
    refs = {}
    data = []

    colcounter = []
    colrefs = set()
    coldata = {}
    
   
    summary = x.get_tree(y)
    
    count = 0
    for element in summary.iter(): # this gets the name. It's the second text element on the page, so we need a count.
        if element.text != None:
            if element.text != '':
                count += 1
                if count == 2:
                    nom = element.text.strip()
                    break
                
    for element in summary.iter(): # This checks where the family expenses kick in
        if element.tag != "pdfxml":
            if element.text != '':
                if ast.literal_eval(element.get("bbox"))[0] in [39.146,52.106]:
                    refs[ast.literal_eval(element.get("bbox"))[1]] = element.text.strip()
                    if "Family Travel Costs" == element.text.strip():
                        family = ast.literal_eval(element.get("bbox"))[1]
                #print element.text,ast.literal_eval(element.get("bbox"))
    #print 'refs',refs
    
    # This gets the column positions for the data
    for element in summary.iter():
        if element.tag != "pdfxml":
            if element.text != '':
                if ast.literal_eval(element.get("bbox"))[1] in refs.keys():
                    if '$' in element.text:
                        colcounter.append(int(ast.literal_eval(element.get("bbox"))[2]))
                        #data[refs[ast.literal_eval(element.get("bbox"))[1]]] = element.text
    
    #print 'colcounter',colcounter
    counter = collections.Counter(colcounter)
    for group in counter.most_common():
        if group[1] > 5: # This removes footnotes stuffing up the column alignment
            colrefs.update([group[0]])
    #print 'colrefs',colrefs
    
    # This creates a string for each data column                    
    for ref in colrefs:
        coldata[ref] = ''

    # This creates the string for each column
    for element in summary.iter():
        if element.tag != "pdfxml":
            if element.text != '':
                for ref in colrefs:
                    if -1 < ref-ast.literal_eval(element.get("bbox"))[2] < 1:
                        if '$' not in element.text:
                            coldata[ref]+=element.text
    #print 'coldata',coldata
    
    # This gets the location of the Family Travel Costs divider and appends Family Travel Costs to 
    # Line items below, excluding Total Expenditure
    for r in refs.keys():
        if r < family:
            if refs[r] != 'Total Expenditure':
                refs[r] = refs[r] + ' - Family'                        

    # This gets the data
    for element in summary.iter():
        if element.tag != "pdfxml":
            if element.text != '':
                for ref in refs.keys():
                    if ast.literal_eval(element.get("bbox"))[1] == ref:
                        for col in coldata.keys():
                            if -1<ast.literal_eval(element.get("bbox"))[2]-col<30: # This checks to see if the data bounding box matches up to the column (with some fat to allow for footnotes)
                                if " " in element.text:
                                    data.append([nom,refs[ref],coldata[col],float(element.text.replace("$","").replace(",","").split(" ")[0])])
                                else:
                                    data.append([nom,refs[ref],coldata[col],float(element.text.replace("$","").replace(",",""))])
    return data,coldata

bigdata = []

for name in sumref.keys():
    pdf = pdfquery.PDFQuery("pdfs/"+name)
    result = get_sum(pdf,sumref[name])
    for part in result[0]:
        bigdata.append(part)
    #df = pd.DataFrame(part,columns=['Name','Type','Period','Amount'])
    # Testing expenditure tally
    #for period in result[1].values():
    #    if not -0.01 < round(df[(df['Type']!='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) - round(df[(df['Type']=='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) < 0.01:
    #        print name,period
df = pd.DataFrame(bigdata,columns=['Name','Type','Period','Amount'])


In [17]:
df['Name']

0       Senator the Hon Joe Ludwig
1       Senator the Hon Joe Ludwig
2       Senator the Hon Joe Ludwig
3       Senator the Hon Joe Ludwig
4       Senator the Hon Joe Ludwig
5       Senator the Hon Joe Ludwig
6       Senator the Hon Joe Ludwig
7       Senator the Hon Joe Ludwig
8       Senator the Hon Joe Ludwig
9       Senator the Hon Joe Ludwig
10      Senator the Hon Joe Ludwig
11      Senator the Hon Joe Ludwig
12      Senator the Hon Joe Ludwig
13      Senator the Hon Joe Ludwig
14      Senator the Hon Joe Ludwig
15      Senator the Hon Joe Ludwig
16      Senator the Hon Joe Ludwig
17      Senator the Hon Joe Ludwig
18      Senator the Hon Joe Ludwig
19      Senator the Hon Joe Ludwig
20      Senator the Hon Joe Ludwig
21      Senator the Hon Joe Ludwig
22      Senator the Hon Joe Ludwig
23      Senator the Hon Joe Ludwig
24      Senator the Hon Joe Ludwig
25      Senator the Hon Joe Ludwig
26      Senator the Hon Joe Ludwig
27      Senator the Hon Joe Ludwig
28      Senator the 

In [20]:
# error in Melissa Park - it overstates her expenses
for name in localpdfs:
    pdf = pdfquery.PDFQuery("pdfs/"+name)
    result = get_sum(pdf,sumref[name])
    df = pd.DataFrame(result[0],columns=['MP/Sen','Type','Period','Amount'])
    display(df)
    # Testing expenditure tally
    for period in result[1].values():
        print round(df[(df['Type']!='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2)
        print round(df[(df['Type']=='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2)
        if not -0.01 < round(df[(df['Type']!='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) - round(df[(df['Type']=='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) < 0.01:
            print name,period

,MP/Sen,Type,Period,Amount
0,The Hon Tony Abbott MP,Parliamentary Travelling Allowance,Expenses From 1 January 2015,24640.00
1,The Hon Tony Abbott MP,Parliamentary Travelling Allowance,Expenses Before 1 January 2015,560.00
2,The Hon Tony Abbott MP,Ministerial Visits,Expenses From 1 January 2015,359612.24
3,The Hon Tony Abbott MP,Office Holder,Expenses From 1 January 2015,0.00
4,The Hon Tony Abbott MP,Private-Plated Vehicle,Expenses From 1 January 2015,6135.37
5,The Hon Tony Abbott MP,COMCAR,Expenses From 1 January 2015,47138.26
6,The Hon Tony Abbott MP,Office Consumables and Services,Expenses From 1 January 2015,8725.30
7,The Hon Tony Abbott MP,Printing and Communications,Expenses From 1 January 2015,38756.53
8,The Hon Tony Abbott MP,Publications,Expenses From 1 January 2015,358.10
9,The Hon Tony Abbott MP,Telecommunications - Usage,Expenses From 1 January 2015,7085.34


566925.0
566925.0
22764.07
22764.07


,MP/Sen,Type,Period,Amount
0,Senator the Hon Eric Abetz,Parliamentary Travelling Allowance,Expenses From 1 January 2015,14772.00
1,Senator the Hon Eric Abetz,Fares,Expenses From 1 January 2015,30370.47
2,Senator the Hon Eric Abetz,Electorate,Expenses From 1 January 2015,3745.45
3,Senator the Hon Eric Abetz,Private-Plated Vehicle,Expenses From 1 January 2015,6817.39
4,Senator the Hon Eric Abetz,COMCAR,Expenses From 1 January 2015,12369.65
5,Senator the Hon Eric Abetz,Cabcharge,Expenses From 1 January 2015,0.00
6,Senator the Hon Eric Abetz,Office Consumables and Services,Expenses From 1 January 2015,11198.55
7,Senator the Hon Eric Abetz,Printing and Communications,Expenses From 1 January 2015,15235.68
8,Senator the Hon Eric Abetz,Additional Printing and Communications,Expenses From 1 January 2015,0.00
9,Senator the Hon Eric Abetz,Publications,Expenses From 1 January 2015,1911.29


162767.52
162767.52
8427.11
8427.11


,MP/Sen,Type,Period,Amount
0,The Hon Dick Adams,Travelling Allowance,Expenses From 1 January 2015,0.00
1,The Hon Dick Adams,Travelling Allowance,Expenses Before 1 January 2015,0.00
2,The Hon Dick Adams,Family Travel Costs,Expenses From 1 January 2015,0.00
3,The Hon Dick Adams,Total Expenditure,Expenses From 1 January 2015,0.00
4,The Hon Dick Adams,Family Travel Costs,Expenses Before 1 January 2015,0.00
5,The Hon Dick Adams,Total Expenditure,Expenses Before 1 January 2015,81.05
6,The Hon Dick Adams,Overseas Travel,Expenses From 1 January 2015,0.00
7,The Hon Dick Adams,Overseas Travel,Expenses Before 1 January 2015,0.00
8,The Hon Dick Adams,Domestic Scheduled Fares,Expenses From 1 January 2015,0.00
9,The Hon Dick Adams,Domestic Scheduled Fares,Expenses Before 1 January 2015,0.00


...

212308.81
212308.81
25305.89
25305.89


,MP/Sen,Type,Period,Amount
0,Senator Nick Xenophon,Parliamentary Travelling Allowance,Expenses From 1 January 2015,12233.00
1,Senator Nick Xenophon,Fares,Expenses From 1 January 2015,12793.52
2,Senator Nick Xenophon,COMCAR,Expenses From 1 January 2015,671.54
3,Senator Nick Xenophon,Cabcharge,Expenses From 1 January 2015,1458.25
4,Senator Nick Xenophon,Other Car Costs,Expenses From 1 January 2015,68.64
5,Senator Nick Xenophon,Parking,Expenses From 1 January 2015,35.45
6,Senator Nick Xenophon,Office Consumables and Services,Expenses From 1 January 2015,938.67
7,Senator Nick Xenophon,Printing and Communications,Expenses From 1 January 2015,174.55
8,Senator Nick Xenophon,Publications,Expenses From 1 January 2015,1430.13
9,Senator Nick Xenophon,Telecommunications - Usage,Expenses From 1 January 2015,6429.10


83086.89
83086.89
10865.24
10865.24


,MP/Sen,Type,Period,Amount
0,Mr Tony Zappia MP,Parliamentary Travelling Allowance,Expenses From 1 January 2015,12534.00
1,Mr Tony Zappia MP,Parliamentary Travelling Allowance,Expenses Before 1 January 2015,0.00
2,Mr Tony Zappia MP,Parliamentary Delegations,Expenses From 1 January 2015,0.00
3,Mr Tony Zappia MP,Parliamentary Delegations,Expenses Before 1 January 2015,-5629.00
4,Mr Tony Zappia MP,Fares,Expenses From 1 January 2015,12040.70
5,Mr Tony Zappia MP,COMCAR,Expenses From 1 January 2015,2593.53
6,Mr Tony Zappia MP,Office Consumables and Services,Expenses From 1 January 2015,4861.11
7,Mr Tony Zappia MP,Printing and Communications,Expenses From 1 January 2015,18560.35
8,Mr Tony Zappia MP,Publications,Expenses From 1 January 2015,966.40
9,Mr Tony Zappia MP,Telecommunications - Usage,Expenses From 1 January 2015,5476.39


106063.39
106063.39
31174.35
31174.35


In [107]:
for name in ['P35_PARKE_Melissa.pdf']:
    pdf = pdfquery.PDFQuery("pdfs/"+name)
    tree = pdf.get_tree(2)
    for element in tree.iter():
        if element.text != None:
            if element.text != '':
                print element.text

1 July to 31 December 2014 
The Hon Melissa Parke MP 
Summary of Parliamentary Expenditure by Period 
Expenses From 
1 July 2014 
Expenses Before 
1 July 2014 
Parliamentary Travelling Allowance 
Fares 
Private-Plated Vehicle 
COMCAR 
Cabcharge 
Other Car Costs 
Office Consumables and Services 
Printing and Communications 
Publications 
Telecommunications - Usage 
Domestic Scheduled Fares 
Total Expenditure 
1 Includes an adjustment of -$25.00 
2 Includes an adjustment of -$25.00 
$7,148.00 
$27,286.76 
1 
$5,010.98 
$1,398.58 
$40.87 
$0.00 
$5,702.22 
$25,704.41 
$1,523.99 
$3,020.24 
$3,341.00 
$3,203.69 
$194.99 2 
$202.00 
$60.44 
$514.03 
$250.98 
$3,556.07 
$597.44 
$0.00 
$0.00 
$134,414.50 
$1,521.08 
$15,613.07 
Travelling Allowance 
Overseas Travel 
$0.00 
$0.00 
Domestic Scheduled Fares 
Charter 
$0.00 
$0.00 
Car Costs 
Office Facilities 
$52,567.47 
$1,976.36 
Office Fit Outs 
$0.00 
$0.00 
Office Administrative Costs 
Telecommunications 
Family Travel Costs 
Page 2 of 22

In [104]:
for element in tree.iter():
    if element.text != None:
        if element.text != '':
            print element.text
            break

The Hon Melissa Parke MP 


In [177]:
# Let's get some values about the doc. This will also give us information about what information to expect.

### First, check what kind of expenses are in there, and whether there are family expenses, 
# and return what pages they are on.

# These are our important categories:
# Travelling Allowance / Parliamentary Travelling Allowance
# Overseas Travel
# Domestic Scheduled Fares / Fares
# Car Costs / Private-Plated Vehicle
# Car Costs / COMCAR
# Car Costs / Cabcharge
# Office Administrative Costs / Office Consumables and Services
# Office Administrative Costs / Printing and Communications
# Office Administrative Costs / Publications
# Telecommunications / Telecommunications - Usage
# Telecommunications / Telecommunications - Residential - Official
# Family Travel Costs / various

cats = [
    "Travelling Allowance / Parliamentary Travelling Allowance",
"Overseas Travel",
"Domestic Scheduled Fares / Fares",
"Car Costs / Private-Plated Vehicle",
"Car Costs / COMCAR",
"Car Costs / Cabcharge",
"Office Administrative Costs / Office Consumables and Services",
"Office Administrative Costs / Printing and Communications",
"Office Administrative Costs / Publications",
"Telecommunications / Telecommunications - Usage",
"Telecommunications / Telecommunications - Residential - Official",
"Family Travel Costs / various",
"Summary of Parliamentary Expenditure by Period / "
]


import ast

pagerefs = {} # This is to record the page refs for the various categories

dics = {} # This holds all our data



for i,element in enumerate(tree.iter()):
    
    # This runs through each element...
    if element.tag != "pdfxml": # excludes the first container element
        if element.text != '': # excludes empty elements
            if element.text != None: # excludes other empty elements
                for c in cats: # This finds what page each of the important categories is on
                    if c.split(" / ")[0] in element.text.strip():
                        for pageno in element.iterancestors(tag="LTPage"):
                            pagerefs.setdefault(c.split(" / ")[0], set()).update([int(pageno.get("page_index"))])

    # This part repeats the same as above, but is for collecting the data.
    if element.tag != "pdfxml": # excludes the first container element
        if element.text != '': # excludes empty elements
            if element.text != None: # excludes other empty elements
                # This part adds all the data elements to the dics disctionary is they line up horizontally.
                #if round(ast.literal_eval(element.get("bbox"))[1],1) in dics[page].keys():
                    #print page,element.text,element.get("bbox")
                #    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)].append(element.text)
                #else:
                    #print page,element.text,element.get("bbox")
                #    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)] = [element.text]
                pass

pagerefs

{'Car Costs': {1, 2, 4, 10, 11, 12, 13, 14, 23, 26},
 'Domestic Scheduled Fares': {2, 4, 8, 9, 21, 23, 25, 30},
 'Family Travel Costs': {2, 4, 21, 23, 30},
 'Office Administrative Costs': {2, 4, 17, 18, 19, 23, 28},
 'Overseas Travel': {2, 4, 6, 7},
 'Summary of Parliamentary Expenditure by Period': {2},
 'Telecommunications': {1, 2, 4, 20, 23, 29},
 'Travelling Allowance': {2, 4, 5, 23, 24}}

Expenditure checks out?
----
EXPENSES BEFORE 1 JULY 2014 
Item tally: 193815.97
Total     : 193815.97
-----
EXPENSES FROM 1 JULY 2014 
Item tally: 15039.78
Total     : 15039.78
-----


In [168]:
pagerefs

{'Car Costs': {'1', '10', '11', '12', '13', '14', '2', '23', '26', '4'},
 'Domestic Scheduled Fares': {'2', '21', '23', '25', '30', '4', '8', '9'},
 'Family Travel Costs': {'2', '21', '23', '30', '4'},
 'Office Administrative Costs': {'17', '18', '19', '2', '23', '28', '4'},
 'Overseas Travel': {'2', '4', '6', '7'},
 'Summary of Parliamentary Expenditure by Period': {'2'},
 'Telecommunications': {'1', '2', '20', '23', '29', '4'},
 'Travelling Allowance': {'2', '23', '24', '4', '5'}}

In [185]:
# Third, let's get how many expenses are in each category.

for cat in pagerefs.keys()[:1]:
    print cat
    for page in pagerefs[cat]:
        if int(page) > 4: # this ignores the first four pages - summary and cover pages
            print page
            tree = pdf.get_tree(page)
            for element in tree.iter():
                if element.text != None:
                    if element.text != '':
                        if element.text.strip() == 'Family':
                            print 'family',ast.literal_eval(element.get('bbox'))
                        if element.text.strip() == 'Private-Plated Vehicle':
                            print 'pp',ast.literal_eval(element.get('bbox'))
                        if element.text.strip() == 'COMCAR':
                            print 'cc',ast.literal_eval(element.get('bbox'))
                        if element.text.strip() == 'Cabcharge':
                            print 'cab',ast.literal_eval(element.get('bbox'))
         
            


Car Costs
10
pp [30.362, 673.077, 140.462, 688.401]
cc [30.362, 297.669, 75.662, 312.993]
11
cc [30.362, 673.077, 75.662, 688.401]
12
cc [30.362, 673.077, 75.662, 688.401]
13
cc [30.362, 673.077, 75.662, 688.401]
cab [30.362, 145.533, 82.406, 160.857]
14
cab [30.362, 673.077, 82.406, 688.401]
23
pp [80.618, 497.177, 169.938, 509.887]
cc [80.618, 480.185, 117.638, 492.895]
cab [80.618, 463.193, 123.398, 475.903]
26
pp [30.362, 673.077, 140.462, 688.401]
cc [30.362, 531.525, 75.662, 546.849]
cab [30.362, 389.973, 82.406, 405.297]


In [ ]:


# Fourt, let's compare the expenses with the totals.

In [18]:
import ast

page = 0
dics = {}
for i,element in enumerate(tree.iter()):
    if element.tag == "LTPage":
        page = element.get("page_index")
        dics[page]={}
    if element.tag != "pdfxml":
        if element.text != '':
            if element.text != None:
                if round(ast.literal_eval(element.get("bbox"))[1],1) in dics[page].keys():
                    print page,element.text,element.get("bbox")
                    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)].append(element.text)
                else:
                    print page,element.text,element.get("bbox")
                    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)] = [element.text]

0 Ms Gai Brodtmann MP  [44.536, 518.031, 266.176, 548.535]
0 Expenditure on Entitlements paid by the  [44.536, 450.271, 372.136, 475.691]
0 Department of Finance  [44.536, 425.863, 231.976, 451.283]
0 1 July to 31 December 2014  [44.536, 377.695, 267.256, 403.495]
0 Final Report  [44.536, 306.843, 135.13, 329.757]
1 1 July to 31 December 2014  [31.658, 567.114, 145.668, 579.884]
1 Ms Gai Brodtmann MP  [31.658, 542.998, 142.478, 558.25]
1 Summary of Parliamentary Expenditure  [278.403, 503.011, 508.143, 520.889]
1 Paid between 1 July and 31 December 2014  [294.243, 483.961, 492.386, 498.008]
1 Ms Gai Brodtmann MP  [31.226, 402.775, 105.106, 412.943]
1 Canberra  [154.059, 402.775, 183.931, 412.943]
1 $708.00  [237.003, 403.313, 260.019, 412.21]
1 $0.00  [289.995, 403.313, 305.927, 412.21]
1 $2,771.03  [335.211, 403.313, 363.512, 412.21]
1 $0.00  [388.995, 403.313, 404.927, 412.21]
1 $7,131.82  [433.059, 403.313, 461.36, 412.21]
1 $66,530.61  [481.011, 403.313, 512.854, 412.21]
1 $18,142.

In [94]:
x = 2

1<x<2.1

True

SelectorSyntaxError: Expected selector, got <DELIM '/' at 0>

In [50]:
x0,y0,x1,y1

'LTTextLineHorizontal'

In [47]:
y.

<Element LTTextLineHorizontal at 0x107b5faa0>

In [114]:
df.groupby('Type').sum()['Amount']

Type
Additional Printing and Communications             2423.19
COMCAR                                          1637429.17
COMCAR - Family                                   32309.36
Cabcharge                                         55918.72
Cabcharge - Family                                  138.55
Car Costs                                             0.00
Charter                                               0.00
Daily Expense Allowance                            9102.00
Domestic Scheduled Fares                              0.00
Domestic Scheduled Fares - Family                998265.87
Electorate                                       320327.86
Family Travel Costs                                   0.00
Fares                                           6468418.72
Ministerial Visits                              2678365.28
Office Administrative Costs                           0.00
Office Consumables and Services                 2742120.88
Office Facilities                              1643

In [22]:
for cat in df['Type'].unique():
    display(cat,df[df['Type'] == cat].groupby('MP/Sen').sum().sort('Amount',ascending=False)[:3])

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


'Parliamentary Travelling Allowance'

,Amount
MP/Sen,
Mr Tony Zappia MP,12534


'Parliamentary Delegations'

,Amount
MP/Sen,
Mr Tony Zappia MP,-5629


...

,Amount
MP/Sen,
Mr Tony Zappia MP,0


'Office Facilities'

,Amount
MP/Sen,
Mr Tony Zappia MP,48030.07


'Office Fit Outs'

,Amount
MP/Sen,
Mr Tony Zappia MP,0


In [130]:
df[df['Type'] == 'SMOS Approved']

,Name,Type,Period,Amount
1496,The Hon Alan Tudge MP,SMOS Approved,Expenses From 1 July 2014,0.00
1497,The Hon Alan Tudge MP,SMOS Approved,Expenses Before 1 July 2014,3027.27
7008,The Hon Michael McCormack MP,SMOS Approved,Expenses From 1 July 2014,2590.00
7024,The Hon Michael McCormack MP,SMOS Approved,Expenses Before 1 July 2014,0.00
7440,The Hon Darren Chester MP,SMOS Approved,Expenses From 1 July 2014,10804.69
7454,The Hon Darren Chester MP,SMOS Approved,Expenses Before 1 July 2014,4910.07
7804,Mr Mark Coulton MP,SMOS Approved,Expenses From 1 July 2014,28272.70
7820,Mr Mark Coulton MP,SMOS Approved,Expenses Before 1 July 2014,6909.10
